In [2]:
import os
import sys
import math
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import torchvision.models as models

from torch.utils.data import DataLoader
from pathlib import Path
from customDataset import *
from tqdm import tqdm
from utilsFn import *
from model import *

# Hyper Parameters

In [3]:

numEpochs = 200
batchSize = 1
lr = 0.01
classes = 2
patchSize = H = W =512
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

homedir = str(Path.home())
trainFile = 'train.csv'
testFile = 'test.csv'
experimentName = time.strftime("%d%b%Y%H%M",time.localtime())

# Dataset

In [4]:
dataset = MammographyDataset(trainFile,homedir,patchSize,mode='train')
testDataset = MammographyDataset(testFile,homedir,patchSize,mode='test')
trainDataset , valDataset = trainValSplit(dataset,val_share=0.1)
trainLoader = DataLoader(trainDataset,batchSize,shuffle=True)
valLoader = DataLoader(valDataset,batchSize,shuffle=True)
#------------- test transformation should be stop-------------#
testLoader = DataLoader(testDataset,batchSize,shuffle=False)

numberOfTrainData = trainDataset.__len__()
numberOfValData = valDataset.__len__()
numberOfTestData =  testDataset.__len__()

total_step=len(trainLoader)


In [5]:
resnet = models.resnet18(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
model = getCustomPretrained(resnet,classes)
criterion = MIL_loss(0.5)
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()),lr=lr)
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad,model.parameters()),lr=lr,weight_decay=0.0001,momentum=0.9)
model = model.to(device)


In [5]:
# dataiter = iter(trainLoader)
# batch,bag_label,bag_img = dataiter.next()
# images = batch[0].to(device)
# labels = bag_label[0].to(device)
# scores , pred = model(images)
# loss = criterion(scores,labels)
# acc = accuracyperImage(pred,labels)
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()
# # print(loss.item())
# print(pred)

In [6]:
# labels.data

In [7]:
# with torch.no_grad():
#     batch,bag_label,bag_img = dataiter.next()
#     images = batch[0].to(device)
#     labels = bag_label[0].to(device)
#     scores , pred = model(images)
#     print(pred)

In [8]:
# scores

In [9]:
def accuracyperImage(pred,labels):
    with torch.no_grad():
        pred_class =  torch.zeros_like(pred)
        pred_class[pred.argmax()]=1
        if(torch.equal(pred_class,labels)):
            return 1
        else:
            return 0

In [10]:
def basiclogging(str):
    logger('*******************MIL CNN*********************','info',str)
    logger('Size of training dataset {}'.format(numberOfTrainData),'info',str)
    logger('Size of Validation dataset {}'.format(numberOfValData),'info',str)
    logger('Size of testing dataset {}'.format(numberOfTestData),'info',str)
    
    if(str=='trainlog'):
        logger('No. of Epochs: {}\n Batch size: {}\n Learning_rate : {}\n Patch size {}*{}\n Step {}'
            .format(numEpochs,batchSize,lr,H,W,total_step),'info',str)
        logger('Epoch\tBatchTime\tAverageBatchTime\tLoss\tAverageLoss\tAccuracy\AverageAccuracy','info',str)
    if(str=='vadLog'):
        logger('Epoch\tBatchTime\tAverageBatchTime\tLoss\tAverageLoss\tAccuracy\AverageAccuracy','info',str)
    if(str=='testlog'):
        logger('Epoch\tAccuracy\tAverageAccuracy\tTruePositive\tTrueNegative\tFalsePositive\tFalseNegative\tPath','info',str)

# Logger

In [11]:
trainLogFile = openfile('./logs/'+experimentName+'/train')
valLogFile = openfile('./logs/'+experimentName+'/validation')
testLogFile = openfile('./logs/'+experimentName+'/test')
setup_logger('trainlog',trainLogFile)
setup_logger('testlog',testLogFile)
setup_logger('vadLog',valLogFile)
basiclogging('trainlog')
basiclogging('testlog')
basiclogging('vadLog')

*******************MIL CNN*********************
Size of training dataset 1998
Size of Validation dataset 223
Size of testing dataset 689
No. of Epochs: 200
 Batch size: 1
 Learning_rate : 0.01
 Patch size 512*512
 Step 1998
Epoch	BatchTime	AverageBatchTime	Loss	AverageLoss	Accuracy\AverageAccuracy
*******************MIL CNN*********************
Size of training dataset 1998
Size of Validation dataset 223
Size of testing dataset 689
Epoch	Accuracy	AverageAccuracy	TruePositive	TrueNegative	FalsePositive	FalseNegative	Path
*******************MIL CNN*********************
Size of training dataset 1998
Size of Validation dataset 223
Size of testing dataset 689
Epoch	BatchTime	AverageBatchTime	Loss	AverageLoss	Accuracy\AverageAccuracy


In [12]:
def adjust_learning_rate(optimizer,epoch,initLR):
    '''Sets the learning rate to the initial LR decayed by 10 every 50 epoch'''
    lr = initLR * (0.1 ** (epoch // 50))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
def save_checkpoint(state,is_best,filename=openfile('./models/'+experimentName+'/checkpoint.pth.tar')):
        torch.save(state,filename)
        if is_best:
            shutil.copyfile(filename,openfile('./models/'+experimentName+'/model_best.pth.tar'))

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [15]:
# from sklearn.metrics import confusion_matrix

In [16]:
# pred = [0,1]
# labels = [0,1]

In [17]:
# import numpy as np
# pred=np.asarray(pred)
# labels = np.asarray(labels)

In [18]:
dataiter = iter(testLoader)
batch,bag_label,path = dataiter.next()

In [19]:
path

('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.416403281812750683720028031170500130104/1.3.6.1.4.1.9590.100.1.2.245063149211255120613007755642780114172/000000.dcm',)

In [20]:
def confusionMatrix(pred,labels):
    tp=0
    tn=0
    fp=0
    fn=0
    
    a=labels.argmax().cpu().numpy()
    b=pred.argmax().cpu().numpy()

    if(a==0 and b==0):
        tn = 1
    elif(a==0 and b==1):
        fp=1
    elif(a==1 and b==0):
        fn=1
    else:
        tp=1
        
    return (tp,fp,fn,tn)
    

In [21]:
def test(testLoader,model,epoch):
    truepositive = AverageMeter()
    truenegative = AverageMeter()
    falsepositive = AverageMeter()
    falsenegative = AverageMeter()
    accuracy = AverageMeter()
    
    model.eval()
    with torch.no_grad():
        for i,(batch,bag_label,path) in enumerate(testLoader):
            images = batch[0].to(device)
            labels = bag_label[0].to(device)

            _,pred = model(images)
            
            acc = accuracyperImage(pred,labels)
            
            tp,fp,fn,tn = confusionMatrix(pred,labels)
            
            truepositive.update(tp)
            truenegative.update(tn)
            falsepositive.update(fp)
            falsenegative.update(fn)
            accuracy.update(acc)
            #logger('Epoch\tAccuracy\tAverageAccuracy\tTruePositive\tTrueNegative\tFalsePositive\tFalseNegative\tPath','info','testlog')
            logger('{0}\t{ac.val:.2f}\t{ac.avg:.6f}\t{tp.sum:.2f}\t{tn.sum:.2f}\t{fp.sum:.2f}\t{fn.sum:.2f}\t{p}'
                   .format(i,ac=accuracy,tp=truepositive,tn=truenegative,
                           fp=falsepositive,fn=falsenegative,p=path),'info','testlog')

                   
    print('Average Accuracy :',accuracy.avg)

In [22]:
def train(train_loader,model,criterion,optimizer,epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    avgaccu =  AverageMeter()
    
    model.train()

    end = time.time()
    for i,(batch,bag_label,_) in enumerate(train_loader):
        data_time.update(time.time()-end)
        images = batch[0].to(device)
        labels = bag_label[0].to(device)
        
        scores,pred = model(images)
        loss = criterion(scores,labels)
        
        # top-k ? accuaracy 
        # for now evaluating normal accuracy
        acc = accuracyperImage(pred,labels)
        
        #loss.item() to get the loss value from loss tensor
        losses.update(loss.item(), images.size(0))
        avgaccu.update(acc,images.size(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time.update(time.time() - end)
        end = time.time()

        logger('[{0}][{1}/{2}]\t{batch_time.val:.6f}\t {batch_time.avg:.6f}\t'
              '{loss.val:.9f}\t{loss.avg:.9f}\t'
              '{acc.val:.9f}\t{acc.avg:.9f}\t'.format(epoch,i,len(train_loader),batch_time=batch_time,
                                                      loss=losses, acc=avgaccu),'info','trainlog')

In [23]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    avgaccu =  AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (batch, bag_label,_) in enumerate(val_loader):
            input= batch[0].to(device)
            target = bag_label[0].to(device)
            
            # compute output
            scores,pred = model(input)
            loss = criterion(scores, target)

            # measure accuracy and record loss
            acc= accuracyperImage(pred, target)
            losses.update(loss.item(), input.size(0))
            avgaccu.update(acc,input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            logger('[{0}/{1}]\t'
                  '{batch_time.val:.6f}\t{batch_time.avg:.6f}\t'
                  '{loss.val:.9f}\t{loss.avg:.9f})\t'
                  '{acc.val:.9f}\t{acc.avg:.9f}'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,acc=avgaccu),'info','vadLog')

        print(' * Acc {acc.avg:.9f}'
              .format(acc=avgaccu))

    return avgaccu.avg

# Analysis What's going wrong



In [24]:
# test(testLoader,model,1)

0	0.00	0.000000	1.00	0.00	0.00	0.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.416403281812750683720028031170500130104/1.3.6.1.4.1.9590.100.1.2.245063149211255120613007755642780114172/000000.dcm',)
1	0.00	0.000000	2.00	0.00	0.00	0.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.100.1.2.125010935311829990124529079264106154392/1.3.6.1.4.1.9590.100.1.2.85952214611170506017891429690540035518/000000.dcm',)
2	0.00	0.000000	3.00	0.00	0.00	0.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.289610447411344525237308079592285912683/1.3.6.1.4.1.9590.100.1.2.22131189612893294827907969600765582967/000000.dcm',)
3	0.00	0.000000	4.00	0.00	0.00	0.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.100.1.2.22378064110931464632702821421529389998/1.3.6.1.4.1.9590.100.1.2.23994906441209206870656672649041512

32	0.00	0.000000	16.00	14.00	0.00	3.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00173_LEFT_MLO/1.3.6.1.4.1.9590.100.1.2.307793166912734364002335835122757098128/1.3.6.1.4.1.9590.100.1.2.116019378912146865822367996522433813122/000000.dcm',)
33	0.00	0.000000	16.00	15.00	0.00	3.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00173_RIGHT_CC/1.3.6.1.4.1.9590.100.1.2.332975382811750735925404835072209681216/1.3.6.1.4.1.9590.100.1.2.164477615412098522241149597592405252179/000000.dcm',)
34	0.00	0.000000	16.00	16.00	0.00	3.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00173_RIGHT_CC/1.3.6.1.4.1.9590.100.1.2.332975382811750735925404835072209681216/1.3.6.1.4.1.9590.100.1.2.164477615412098522241149597592405252179/000000.dcm',)
35	0.00	0.000000	16.00	17.00	0.00	3.00	('/home/himanshu/CuratedDDSM/Test/Mass/CBIS-DDSM/Mass-Test_P_00173_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.246073522010210423031364901130375270352/1.3.6.1.4.1.9590.100.1.2.95930514311143196

KeyboardInterrupt: 

In [25]:
# validate(valLoader,model,criterion)

[0/223]	0.597227	0.597227	3.747858524	3.747858524)	0.000000000	0.000000000
[1/223]	1.059858	0.828542	0.083802566	1.915830545)	1.000000000	0.500000000
[2/223]	0.806103	0.821062	4.221531868	2.684397653)	0.000000000	0.333333333
[3/223]	0.828717	0.822976	5.133491039	3.296670999)	0.000000000	0.250000000
[4/223]	0.730241	0.804429	0.036272824	2.644591364)	1.000000000	0.400000000
[5/223]	1.621907	0.940675	2.967804432	2.698460209)	0.000000000	0.333333333
[6/223]	1.383396	1.003921	2.734069586	2.703547263)	0.000000000	0.285714286
[7/223]	3.866378	1.361728	0.061092377	2.373240402)	1.000000000	0.375000000


KeyboardInterrupt: 

In [26]:
# train(trainLoader,model,criterion,optimizer,1)

[1][0/1998]	0.741024	 0.741024	3.090146780	3.090146780	0.000000000	0.000000000	
[1][1/1998]	0.794239	 0.767632	1.414602757	2.252374768	1.000000000	0.500000000	
[1][2/1998]	0.668290	 0.734518	2.286785603	2.263845046	0.000000000	0.333333333	
[1][3/1998]	2.363228	 1.141695	-0.000000000	1.697883785	1.000000000	0.500000000	
[1][4/1998]	1.622602	 1.237877	-0.000000000	1.358307028	0.000000000	0.400000000	
[1][5/1998]	3.130916	 1.553383	-0.000000000	1.131922523	1.000000000	0.500000000	
[1][6/1998]	1.738377	 1.579811	-0.000000000	0.970219306	0.000000000	0.428571429	
[1][7/1998]	2.739412	 1.724761	-0.000000000	0.848941892	0.000000000	0.375000000	


KeyboardInterrupt: 

# Training Loop

In [ ]:
is_best=0

for epoch in range(numEpochs):
    
    adjust_learning_rate(optimizer,epoch+1,lr)
    
    train(trainLoader,model,criterion,optimizer,epoch+1)
    
    pause(strg='training pause')
    
    acc = validate(valLoader,model,criterion)
    
    is_best = acc > best
    
    best_acc = max(acc,best_acc)
    
    save_checkpoint({
        'epoch':epoch+1,
        'state_dict':model.state_dict(),
        'best_acc': best_acc,
        'optimizer':optimizer.state_dict(),
    },is_best)
    